### 🏠 LH 공사 임대/분양 공고 RAG 파이프라인
### **목표**: 사용자 질문에 대해 Multi-Query, Hybrid Search (Dense + Sparse), Cross-Encoder Rerank를 적용하여 LH 공고 문서에서 정확한 답변을 생성합니다.

## 1. 라이브러리 임포트 및 환경 설정

In [1]:
import os
import json
import asyncio
import time
from datetime import datetime
from typing import List, Dict

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# LangChain Core
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# LangChain LLM & Embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# VectorStore
from langchain_postgres import PGVector
from langchain_postgres.vectorstores import DistanceStrategy

# Sparse Search
from rank_bm25 import BM25Okapi

# CrossEncoder
from sentence_transformers import CrossEncoder

# Pydantic
from pydantic import BaseModel, Field

# PostgreSQL (asyncpg는 비동기 함수 내에서 사용됨)
import asyncpg

# API 키 및 DB 설정 로드
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")
if not OPENAI_API_KEY or OPENAI_API_KEY == "YOUR_API_KEY_HERE":
    # ⚠️ 노트북에서 환경변수 로드가 안 될 경우, 여기에 직접 API 키를 입력하세요.
    # OPENAI_API_KEY = "sk-..."
    print("⚠️ OPENAI_API_KEY 환경변수를 설정하거나 코드에 직접 입력해 주세요.")
    # raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

# -----------------------
# DB 설정
# -----------------------
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "skn19_3rd_proj",
    "user": "rag_user",
    "password": "skn19"
}

CONNECTION_STRING = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# -----------------------
# Embedding & LLM
# -----------------------
EMBEDDING_MODEL = "text-embedding-3-small"
embedding_function = OpenAIEmbeddings(model=EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY)
LLM = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=OPENAI_API_KEY)

# 다중 질의 생성용 LLM
QUERY_LLM = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=OPENAI_API_KEY)

# -----------------------
# Dense Retriever (PGVector)
# -----------------------
VECTORSTORE = PGVector(
    connection=CONNECTION_STRING,
    embeddings=embedding_function,
    collection_name="document_chunks",
    distance_strategy=DistanceStrategy.COSINE,
)
dense_retriever = VECTORSTORE.as_retriever(search_kwargs={"k": 10})

# -----------------------
# CrossEncoder
# -----------------------
RERANKER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
RERANKER = CrossEncoder(RERANKER_MODEL)

print("✅ 라이브러리 로드 및 초기 설정 완료")

c:\Users\Playdata\anaconda3\envs\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 라이브러리 로드 및 초기 설정 완료


## 2. Pydantic 모델 및 프롬프트 정의

In [2]:
# Pydantic Output
class AnswerResponse(BaseModel):
    answer: str = Field(description="사용자 질문에 대한 답변 (마크다운)")
    source_documents: List[Dict[str, str]] = Field(description="참고 문서 정보")
    confidence: str = Field(default="높음", description="답변 신뢰도: 높음/중간/낮음")
    queries_used: List[str] = Field(default=[], description="검색에 사용된 질의 목록")

# Multi-Query Prompt
MULTI_QUERY_PROMPT = """당신은 LH 공사 임대/분양 공고 검색을 돕는 AI 어시스턴트입니다.
사용자의 질문을 다양한 관점에서 재작성하여 검색 성능을 높이세요.

원본 질문에 대해 3개의 다른 버전을 생성하세요:
1. 동의어나 유사 표현을 사용한 버전
2. 더 구체적이거나 상세한 버전  
3. 더 일반적이거나 넓은 범위의 버전

규칙:
- 각 질문은 한 줄에 하나씩 작성
- 번호나 기호 없이 질문만 작성
- 원본 질문의 의도를 유지
- LH, 임대주택, 분양주택 관련 용어 활용

원본 질문: {question}

변환된 질문들:"""

# RAG System Prompt
SYSTEM_PROMPT = """당신은 LH(한국토지주택공사)의 임대주택 및 분양주택 공고 전문 상담사입니다.

## 역할
- 사용자의 질문에 대해 제공된 검색 문서를 기반으로 정확하고 친절하게 답변합니다.
- LH 공고의 자격 조건, 신청 방법, 일정, 필요 서류 등에 대해 안내합니다.

## 답변 원칙
1. **정확성**: 반드시 제공된 문서 내용만을 기반으로 답변하세요. 문서에 없는 내용은 추측하지 마세요.
2. **명확성**: 복잡한 자격 조건이나 절차는 단계별로 나누어 설명하세요.
3. **완전성**: 질문과 관련된 모든 중요 정보(일정, 조건, 주의사항 등)를 포함하세요.
4. **친절함**: 전문 용어는 쉽게 풀어서 설명하고, 추가로 알아두면 좋은 정보도 안내하세요.

## 답변 형식
- 마크다운 형식으로 작성하세요.
- 중요한 날짜, 금액, 자격 조건은 **강조** 표시하세요.
- 여러 항목이 있을 경우 목록이나 표로 정리하세요.
- 답변 마지막에 관련 문의처나 추가 확인 방법을 안내하세요.

## 주의사항
- 문서에서 답을 찾을 수 없는 경우, 솔직하게 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 안내하세요.
- 공고마다 조건이 다를 수 있으므로, 해당 공고의 세부 내용을 반드시 확인하도록 안내하세요.
- 신청 기한이 지난 공고인 경우 이를 명시하세요."""

## 3. 핵심 함수 정의 (비동기 포함)

In [3]:
# -----------------------
# Sparse Retriever (BM25) 로드
# -----------------------
async def load_sparse_retriever():
    """
    DB에서 모든 청크를 로드하여 BM25 객체를 초기화합니다.
    (노트북 세션에서 한 번만 실행하는 것이 효율적입니다.)
    """
    conn = await asyncpg.connect(**DB_CONFIG)
    try:
        rows = await conn.fetch("""
            SELECT dc.chunk_text, a.title, a.category, a.region, a.notice_type, dc.metadata
            FROM document_chunks dc
            JOIN announcements a ON dc.announcement_id = a.id
        """)
        documents = []
        corpus = []
        for r in rows:
            metadata = json.loads(r["metadata"]) if isinstance(r["metadata"], str) else r["metadata"]
            metadata.update({
                "title": r["title"],
                "category": r["category"],
                "region": r["region"],
                "notice_type": r["notice_type"]
            })
            doc = Document(page_content=r["chunk_text"], metadata=metadata)
            documents.append(doc)
            corpus.append(r["chunk_text"].split())
        bm25 = BM25Okapi(corpus)
        return bm25, documents
    finally:
        await conn.close()

# -----------------------
# Multi-Query 생성
# -----------------------
def generate_multi_queries(question: str, num_queries: int = 3) -> List[str]:
    """원본 질문을 여러 개의 다른 표현으로 변환합니다."""
    prompt = ChatPromptTemplate.from_template(MULTI_QUERY_PROMPT)
    chain = prompt | QUERY_LLM | StrOutputParser()
    result = chain.invoke({"question": question})
    generated_queries = [q.strip() for q in result.strip().split("\n") if q.strip()]
    all_queries = [question] + generated_queries[:num_queries]
    
    print(f"  📝 생성된 질의 ({len(all_queries)}개):")
    for i, q in enumerate(all_queries):
        prefix = "원본" if i == 0 else f"변환{i}"
        print(f"     [{prefix}] {q}")
    print()
    
    return all_queries

# -----------------------
# CrossEncoder Rerank
# -----------------------
def rerank_documents(question: str, docs: List[Document], top_k: int = 5) -> List[Document]:
    """CrossEncoder로 문서 재정렬"""
    if not docs:
        return []
    
    pairs = [(question, doc.page_content) for doc in docs]
    scores = RERANKER.predict(pairs)
    
    # 점수와 문서를 묶어 점수 내림차순으로 정렬
    reranked = [doc for _, doc in sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)]
    return reranked[:top_k]

# -----------------------
# Multi-Query Hybrid Search
# -----------------------
async def multi_query_search(question: str, bm25_data, top_k_per_query: int = 5) -> List[Document]:
    """다중 질의로 Dense + Sparse 검색을 수행합니다."""
    queries = generate_multi_queries(question)
    bm25, sparse_docs = bm25_data
    all_docs = []
    
    for query in queries:
        # Dense Search
        dense_results = dense_retriever.invoke(query)
        all_docs.extend(dense_results[:top_k_per_query])
        
        # Sparse Search (BM25)
        tokenized_query = query.split()
        bm25_scores = bm25.get_scores(tokenized_query)
        sparse_with_scores = sorted(
            zip(sparse_docs, bm25_scores), 
            key=lambda x: x[1], 
            reverse=True
        )[:top_k_per_query]
        all_docs.extend([doc for doc, _ in sparse_with_scores])
    
    # 중복 제거
    seen_texts = set()
    unique_docs = []
    for doc in all_docs:
        if doc.page_content not in seen_texts:
            unique_docs.append(doc)
            seen_texts.add(doc.page_content)
    
    return unique_docs, queries

# -----------------------
# RAG Response 생성 보조 함수
# -----------------------
def format_source_info(docs: List[Document]) -> List[Dict[str, str]]:
    """답변 출력 시 참고할 문서 정보를 포맷팅"""
    info = []
    seen = set()
    for doc in docs:
        title = doc.metadata.get("title", "N/A")
        if title not in seen:
            seen.add(title)
            info.append({
                "공고명": title,
                "분류": "임대" if doc.metadata.get("category") == "lease" else "분양",
                "지역": doc.metadata.get("region", "N/A"),
                "공고유형": doc.metadata.get("notice_type", "N/A"),
            })
    return info

def format_context(docs: List[Document]) -> str:
    """LLM 입력에 사용할 문서 텍스트를 포맷팅"""
    context_parts = []
    for i, doc in enumerate(docs, 1):
        meta = doc.metadata
        header = f"[문서 {i}]"
        header += f" 공고명: {meta.get('title', 'N/A')}"
        header += f" | 분류: {'임대' if meta.get('category') == 'lease' else '분양'}"
        header += f" | 지역: {meta.get('region', 'N/A')}"
        context_parts.append(f"{header}\n{doc.page_content}")
    return "\n\n---\n\n".join(context_parts)

def analyze_question_type(question: str) -> str:
    """질문 유형 분석에 따른 LLM 지시사항 추가"""
    additional_instructions = []
    
    if any(kw in question for kw in ["자격", "조건", "대상", "누가", "신청할 수 있"]):
        additional_instructions.append("- 자격 조건을 구체적으로 나열하고, 각 조건의 세부 기준(소득, 나이, 거주지 등)을 명시하세요.")
    # ... (생략된 기존 로직 유지)
    if any(kw in question for kw in ["일정", "기간", "언제", "마감", "접수"]):
        additional_instructions.append("- 모든 관련 일정(공고일, 접수기간, 당첨자 발표일, 계약일 등)을 시간순으로 정리하세요.")
    if any(kw in question for kw in ["신청", "방법", "어떻게", "절차", "서류"]):
        additional_instructions.append("- 신청 절차를 단계별로 설명하고, 필요한 서류와 준비물을 목록으로 정리하세요.")
    if any(kw in question for kw in ["비용", "금액", "보증금", "월세", "분양가", "얼마"]):
        additional_instructions.append("- 금액 정보는 정확한 숫자로 제공하고, 가능하면 표로 정리하세요.")
    if any(kw in question for kw in ["차이", "비교", "vs", "어떤 것이"]):
        additional_instructions.append("- 비교 대상들의 차이점을 표로 정리하여 한눈에 볼 수 있게 하세요.")
    
    if additional_instructions:
        return "\n\n## 이 질문에 대한 추가 지시사항\n" + "\n".join(additional_instructions)
    return ""

def create_rag_response(question: str, docs: List[Document], queries_used: List[str] = None) -> AnswerResponse:
    """RAG 답변 생성"""
    if not docs:
        return AnswerResponse(
            answer="죄송합니다. 질문과 관련된 공고 문서를 찾지 못했습니다.\n\n"
                    "다음을 시도해 보세요:\n"
                    "- 다른 키워드로 질문해 주세요\n"
                    "- LH 청약센터(https://apply.lh.or.kr)에서 직접 검색해 보세요\n"
                    "- LH 고객센터(1600-1004)로 문의해 주세요",
            source_documents=[],
            confidence="낮음",
            queries_used=queries_used or []
        )
    
    context_text = format_context(docs)
    additional_instructions = analyze_question_type(question)
    final_system_prompt = SYSTEM_PROMPT + additional_instructions
    
    user_prompt = f"""## 검색된 공고 문서

{context_text}

---

## 사용자 질문
{question}

위 문서들을 참고하여 사용자의 질문에 답변해 주세요."""

    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(content=final_system_prompt),
        ("user", user_prompt)
    ])
    
    messages = prompt_template.format_messages()
    llm_output = LLM.invoke(messages)
    
    confidence = "높음" if len(docs) >= 3 else "중간" if len(docs) >= 1 else "낮음"
    
    response = AnswerResponse(
        answer=llm_output.content,
        source_documents=format_source_info(docs),
        confidence=confidence,
        queries_used=queries_used or []
    )
    return response

# -----------------------
# 통합 실행 함수
# -----------------------
async def run_question_rag(
    question: str, 
    bm25_data, 
    q_number: int = None,
    show_context: bool = False,
    show_timing: bool = True,
    use_multiquery: bool = True
):
    """
    LH RAG 파이프라인 (Multi-Query + Hybrid + Rerank) 실행 및 결과 출력
    """
    start_time = time.time()
    
    print("\n" + "="*100)
    q_label = f"질문 {q_number}" if q_number else "질문"
    print(f"🔍 {q_label}: {question}")
    print(f"⏰ 실행 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"🔄 Multi-Query: {'활성화' if use_multiquery else '비활성화'}")
    print("="*100 + "\n")
    
    # Step 1: 문서 검색
    print("📚 [Step 1/2] 관련 문서 검색 중...")
    search_start = time.time()
    
    if use_multiquery:
        # Multi-Query Hybrid Search + Rerank
        candidate_docs, queries = await multi_query_search(question, bm25_data, top_k_per_query=5)
        docs = rerank_documents(question, candidate_docs, top_k=5)
    else:
        # 단일 질의 Hybrid Search + Rerank (기존 로직)
        queries = [question]
        dense_docs = dense_retriever.invoke(question)
        bm25, sparse_docs = bm25_data
        tokenized_query = question.split()
        bm25_scores = bm25.get_scores(tokenized_query)
        sparse_with_scores = sorted(zip(sparse_docs, bm25_scores), key=lambda x: x[1], reverse=True)[:5]
        combined = dense_docs[:5] + [doc for doc, _ in sparse_with_scores]
        
        seen = set()
        unique = []
        for doc in combined:
            if doc.page_content not in seen:
                unique.append(doc)
                seen.add(doc.page_content)
        
        docs = rerank_documents(question, unique, top_k=5)
        print(f"  🔍 검색된 후보 문서: {len(unique)}개")
    
    search_time = time.time() - search_start
    
    if not docs:
        print("⚠️  관련 문서를 찾지 못했습니다.\n")
        print("💡 TIP: 다른 키워드로 검색하거나 LH 청약센터를 직접 방문해 주세요.")
        print("="*100 + "\n")
        return None
    
    print(f"✅ Rerank 후 최종 문서: {len(docs)}개 ({search_time:.2f}초)\n")
    
    if show_context:
        print("📄 검색된 문서 미리보기:")
        print("-"*80)
        for i, doc in enumerate(docs, 1):
            title = doc.metadata.get("title", "N/A")
            preview = doc.page_content[:150].replace("\n", " ") + "..."
            print(f"  [{i}] {title}")
            print(f"      {preview}\n")
        print("-"*80 + "\n")
    
    # Step 2: 답변 생성
    print("💭 [Step 2/2] 답변 생성 중...")
    generation_start = time.time()
    response = create_rag_response(question, docs, queries_used=queries)
    generation_time = time.time() - generation_start
    print(f"✅ 답변 생성 완료 ({generation_time:.2f}초)\n")
    
    total_time = time.time() - start_time
    
    # 답변 출력
    print("="*100)
    print("### 💬 답변 ###")
    print("="*100 + "\n")
    
    q_prefix = f"Q{q_number}. " if q_number else ""
    print(f"**{q_prefix}{question}**\n")
    print(response.answer)
    
    # 참고 문서 출력
    print("\n" + "-"*100)
    print(f"### 📚 참고 문서 ({len(response.source_documents)}건) | 신뢰도: {response.confidence} ###")
    print("-"*100 + "\n")
    
    if response.source_documents:
        print("| 순번 | 공고명 | 분류 | 지역 | 공고유형 |")
        print("| :---: | :--- | :---: | :---: | :---: |")
        for i, doc in enumerate(response.source_documents, 1):
            title = doc['공고명'][:40] + "..." if len(doc['공고명']) > 40 else doc['공고명']
            print(f"| {i} | {title} | {doc['분류']} | {doc['지역']} | {doc.get('공고유형', 'N/A')} |")
    else:
        print("참고 문서 없음")
    
    # 사용된 질의 출력
    if use_multiquery and response.queries_used:
        print("\n" + "-"*100)
        print("### 🔄 검색에 사용된 질의 ###")
        print("-"*100)
        for i, q in enumerate(response.queries_used):
            prefix = "원본" if i == 0 else f"변환{i}"
            print(f"  [{prefix}] {q}")
    
    if show_timing:
        print("\n" + "-"*100)
        print("### ⏱️ 처리 시간 ###")
        print("-"*100)
        print(f"  • 문서 검색: {search_time:.2f}초")
        print(f"  • 답변 생성: {generation_time:.2f}초")
        print(f"  • 총 소요 시간: {total_time:.2f}초")
    
    print("\n" + "="*100)
    print("💡 TIP: 더 자세한 내용은 LH 청약센터(https://apply.lh.or.kr) 또는")
    print("       LH 고객센터(☎ 1600-1004)에서 확인하세요.")
    print("="*100 + "\n")
    
    return response

## 4. BM25 데이터 초기 로드 (1회 실행)

In [4]:
print("⏳ BM25 Sparse Retriever 데이터 로드 중...")
try:
    # load_sparse_retriever는 비동기 함수이므로 await로 호출해야 합니다.
    # 주피터 노트북 환경에서는 최상위 레벨 await가 지원될 수 있지만,
    # 그렇지 않은 경우 asyncio.run() 또는 await를 사용하는 것이 안전합니다.
    # 이 환경에서는 await를 사용합니다.
    bm25_data = await load_sparse_retriever()
    print(f"✅ BM25 로드 완료. 문서 수: {len(bm25_data[1])}개")
except Exception as e:
    print(f"❌ BM25 로드 실패. DB 연결을 확인하세요: {e}")
    bm25_data = None # 실패 시 테스트 실행 방지

⏳ BM25 Sparse Retriever 데이터 로드 중...
✅ BM25 로드 완료. 문서 수: 20352개


## 5. 테스트 실행


In [5]:
# 테스트 질문 정의
test_questions = [
    "LH 행복주택 청년 대상 조건은 무엇인가요?",
    "신혼부부 특별공급 자격 조건이 궁금합니다.",
    "영구임대주택과 국민임대주택의 차이점은?",
    "LH 청약 시 필요한 서류는 무엇인가요?",
]

# Multi-Query를 활성화하여 첫 번째 질문 테스트
if bm25_data:
    question1 = test_questions[0]
    result1 = await run_question_rag(
        question1, 
        bm25_data, 
        q_number=1, 
        show_context=True, 
        use_multiquery=True
    )
    

# Multi-Query를 비활성화하여 두 번째 질문 테스트 (비교를 위해)
if bm25_data:
    question2 = test_questions[1]
    result2 = await run_question_rag(
        question2, 
        bm25_data, 
        q_number=2, 
        show_context=False, 
        use_multiquery=False
    )


# 나머지 질문 일괄 테스트 (함수 정의는 기존 main 로직을 따라 복수 실행 함수는 제외하고, 개별 실행만 남겨둠)
print("\n" + "--- 나머지 질문 테스트 ---")
if bm25_data:
    for i, q in enumerate(test_questions[2:], 3):
        await run_question_rag(q, bm25_data, q_number=i, use_multiquery=True)
else:
    print("BM25 데이터 로드가 실패하여 테스트를 건너뜁니다.")


🔍 질문 1: LH 행복주택 청년 대상 조건은 무엇인가요?
⏰ 실행 시간: 2025-11-22 20:48:53
🔄 Multi-Query: 활성화

📚 [Step 1/2] 관련 문서 검색 중...
  📝 생성된 질의 (4개):
     [원본] LH 행복주택 청년 대상 조건은 무엇인가요?
     [변환1] LH 행복주택의 청년 대상 자격 요건은 어떻게 되나요?
     [변환2] LH에서 제공하는 행복주택의 청년층을 위한 조건에 대해 자세히 알고 싶습니다.
     [변환3] LH 임대주택의 청년 대상 정책과 관련된 정보를 알고 싶습니다.

✅ Rerank 후 최종 문서: 5개 (5.47초)

📄 검색된 문서 미리보기:
--------------------------------------------------------------------------------
  [1] 화성비봉 A-3블록 가정어린이집 운영예정자 모집공고
      ## 화성비봉 A-3블록 행복주택 LH 가정어린이집 운영예정자모집 공고  공공주택특별법 제 조의 및 영유아보육법 에 따라 화성시 비봉면  삼화리 구포리 일원 화성비봉 공공주택지구 내에 위치한 화성비봉  블록 행복주택 가정어린이집의 설치 및 운영을 위한 운영예정자를  다...

  [2] 2025년 청년 전세임대 1순위 입주자 수시모집
      대상자의 귀책사유로 전세계약이 해제될 경우 기납부한 계약금은 주택소유자에게 귀속될 수 있으니  반드시 LH 해당 지역본부 승인 후 계약)  - 입주대상자는 지원 대상 주택에 전입신고 후 확정일자를 받는 등의 대항력 유지의무가 있으며,  주민등록 미 전입 또는 무단이전 ...

  [3] 성남신촌 A1블록 통합공공임대주택 입주자 모집공고
      - 우리공사에서 제공하는 전화 및 방문상담 등은 신청자의 이해를 돕기 위한 것으로 **공고문을 충분히 숙지하시고 신청**  하시기 바라며, **미확인으로 인한 불이익에 대해 공사에 이의를 제기하실 수 없습니다.**  - **[당첨 이후 연락처(주